<a href="https://colab.research.google.com/github/alissonjcjk/Projeto-A.M/blob/main/Projeto_AM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
Projeto A.M

/content/Projeto-A.M
